In [1]:
%%capture capt
!pip install matplotlib
!pip install pandas
!pip install seaborn
!pip install scikit-learn
!pip install chess
!pip install tensorflow
!pip install tensorflow_addons

In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import time

import warnings
warnings.filterwarnings("ignore")

import chess

import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import datasets, layers, models
from tensorflow_addons.metrics import RSquare
from tensorflow.keras import regularizers

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("GPU is", "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE")

2023-07-02 09:55:46.787024: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-02 09:55:46.832109: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-02 09:55:46.833262: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-02 09:55:47.616823: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Version:  2.12.0
Eager mode:  True
GPU is NOT AVAILABLE


# Functions to represent boards by matrices

In [3]:
%run board_representation.ipynb

# Text-based play

### Functions

In [4]:
def board_to_matrix(board):
    fen = board.fen()
    return fen_to_matrix(fen)

def evaluate_position(board, model):
    board_matrix = board_to_matrix(board)
    evaluation = model.predict(np.array([board_matrix]), verbose=0)
    return evaluation[0, 0]

def get_all_attainable_positions(board, model):
    legal_moves = list(board.legal_moves)
    matrices = {}
    
    for move in legal_moves:
        board.push(move)
        matrices[move] = board_to_matrix(board)
        board.pop()
    return matrices

def evaluate_attainable_positions(board, model):
    matrices = get_all_attainable_positions(board, model)
    X = np.array(list(matrices.values()))
    evaluations = model.predict(X, verbose=0)
    dict_eval = {}
    for i, key in enumerate(matrices.keys()):
        dict_eval[key] = evaluations[i, 0]
    return dict_eval

def find_best_move(board, model):
    dict_evals = evaluate_attainable_positions(board, model)
    if board.turn:
        # white to play
        best_move = max(dict_evals, key=dict_evals.get)
        return best_move, dict_evals[best_move]
    # black to play
    best_move = min(dict_evals, key=dict_evals.get)
    return best_move, dict_evals[best_move]

def play_best_move(board, model, return_eval=True):
    best_move, best_eval = find_best_move(board, model)
    board.push(best_move)
    return best_eval

### Play

In [5]:
model_path = 'model.h5'
model = tf.keras.models.load_model('model.h5')

In [6]:
def play(board, model=model):
    eval = play_best_move(board, model)
    print(f"Evaluation = {round(eval, 1)}")
    display(board)

##### Game

In [8]:
board = chess.Board()